In [1]:
import bs4
import pandas as pd
import requests
import cx_Oracle
import pymysql
import my_utils as mu
from tqdm import tqdm
import numpy as np
import time

# riot api master 구간 데이터 가져와서

# 첫번째 df는 match_id, matches, timeline

# matches 컬럼을 이용 새로운 match_df 만들기

# match df - 컬럼

# gameId, gameDuration, gameVersion, summonerName, sommonerLevel, participantId, championName, champExperience, teamPosition, teamId, win, kills, deaths, assists, totalDamageDealtToChampions, totalDamageTaken

# 테이블 이름 - lol_matches

# insert문 작성해서 넣기


In [2]:
riot_api_key = 'RGAPI-25d88498-9e65-44bb-b80b-7738d7795fee'

In [3]:
url = f'https://kr.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5?api_key={riot_api_key}'

In [4]:
res = requests.get(url).json()

In [5]:
res.keys()

dict_keys(['tier', 'leagueId', 'queue', 'name', 'entries'])

In [6]:
name_lst = []
for i in range(len(res['entries'])):
    name_lst.append(res['entries'][i]['summonerName'])
name_lst=name_lst[:100]    

In [7]:
name_lst

['민채 젖통 핥짝',
 'EH로EH로',
 '밤에올려다본하늘',
 '라칸불',
 '보보캐',
 '조유리즈영원하자',
 '호카게 지망쉔',
 '32ms from china',
 '려 태',
 '네이스좋아요',
 '간토 공방',
 '슬픈솔랭',
 'Counterblow',
 '삶은계란토마토',
 '이기는법모르는자',
 '무지성 점프',
 'hunnami',
 '서폿쿤 스키다요',
 '그마까지폐관수련',
 'un accord',
 '빠송2',
 'Elderwood Azir',
 '타선장',
 '김범수급 높 설치',
 'Xquare',
 '뉴진스 오마이갓',
 '5목눈이',
 '실력은점수로증명',
 'open ff',
 'Roh Min Kyu',
 '유미원챔공주등장',
 '         지잘알 ',
 '북방의 비둘기  ',
 '유혜리',
 'NoFFNoGG',
 '유지사',
 '우리라인은게복치',
 '냄진우',
 '한 를',
 '올뮤트로 게임',
 '봇 좀 와줘',
 '오레오승즈',
 '거 괴',
 '질 장',
 '팀운상위99퍼계정',
 'Teien',
 'paoqiganqingba',
 '예린은귀엽습니다',
 '온 봄',
 'Haiyesi',
 'QAQQWQOVO',
 '현민짱와쿠와쿠',
 '사쿠라 치요노 오',
 '규 모 띠',
 '하민주',
 '보이지 않는 것들',
 '신이훈',
 '바이올렛',
 '20011010',
 '건전한소환사1999',
 'Snowfly',
 'RiverFlowsInY',
 'Rey is Ricky',
 '헉캬흠',
 '15ggzzzTuT',
 'malzaG',
 'JOHANSON',
 '냥 챠',
 '부족해도잘할게',
 '맵시왕',
 '규스오',
 '오늘도 고생핸',
 '허허맨',
 '노약자석양보안함',
 '침묵과긍정',
 '머라고다시말해바',
 '태성고 태기새',
 '세체뽀아니라고요',
 'T1 ADC Test',
 '1wancibeishang',
 'kakao',
 'MYPRIDE',
 '현자 시간',
 '협곡등반가',
 'Tw

In [8]:
mu.get_puuid(name_lst[0])

'QH_dd_7Qq3a0DujOsGZcUaEoSlVdkz0_2B6r7PeclX9k6qqo3xnAxLPjLgnAE-Q2NpAtMhPVZ1nJpQ'

In [10]:
puuid_lst=[]
for i in tqdm(range(len(name_lst))):
    try:
        puuid_lst.append(mu.get_puuid(name_lst[i]))
    except:
        continue

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:27<00:00,  3.65it/s]


In [11]:
puuid_lst

['bJ5PeVBiJlykk45A7zMNastT7BAQRFQ4vvyqC0S_mj0_UiIGVYdAKY6jIb2QAm8djzkOnIC74a1ccw',
 'qxt0LVnVF4PriHPUUf-BFC-BpGIb4wjnhsElv-llqTU1TejS8LS76i9nPtmeFtHq5KvGzBziCQ3TnA',
 '1-1XcXlqTVn-bsL5XrtVWzT35kEPMHsGzG34uUJzqkFbrOsfRJ49DVMBdCp6PNzt-fXQbsoLX-IlQw',
 '1TLlS39-IkH0-Bys78EYqQlR-kRIQaeiS41IBeCmF1uy5DIfEtDFHus-IyPBiaD1U99-LvrhSPdLmg',
 'IcI6rhquNGDc3YVfmppDH7dewsx_pmXOfv1FHfnr38C_XDWHMgzs8jJeiUZDYnHigM2Fn480ULnV1Q',
 '8zVzHN3GYhLBpegaw42jPTr--jAczOBoo-jNsdF0sqzI3DDqqZ62-PQnQ-z8LGhBD54Sbyh6u9j0Lg',
 'LF66Ce6KOfFB7ZC1sUTBN_voNc8Cx00BZo2Nak8lv4wCZ3aP9tqGAT0ZsbzzX5YJPdCP8qfDntPHdg',
 'VUkNi1sH0da0p-O2bMGauIAX5TdaCwUQ4Q3MKNoTzSkVot2j4S9JKvUg6T7Gwza6nhIfk3aEW2s7cA',
 'kGNXUyh0eqPlOMhtaltTDlgf-3cV_ucxCekPIr43-5cPQHyB4j68pZmvPBTpTVR9hLSP6FchreMVMQ',
 'GnNN_KOpRKOrL604Hq5S3aq1QhiuN1EDJgIsdivSBGoadVDdQevRBbiqpTUik_HL7E6_7wJtV16EFA',
 'kJ1k8LpAU72i8mIZ7MSIRFYxnPw925KAt6tyDxDBCN0G20fl18Ks976MZMGCEBtyHU_qqSR4sr9Jmw',
 'hyQ2_CupGV-X6xQUGSSJsYXiQJNnTN8hVptIdtDW8OkYZRm6FBB7cdRtNEC-ImC-Xi9OpVjvWwgeSw',
 'RU

In [74]:
match_ids= []
for i in tqdm(range(len(puuid_lst))):
    try:
        match_ids.append(mu.get_match_id(puuid_lst[i],2))
    except:
        continue

100%|██████████████████████████████████████████████████████████████████████████████████| 97/97 [00:29<00:00,  3.34it/s]


In [97]:
matches = []
timeline = []
for i in tqdm(range(len(match_ids))):
    try:
        matches.append(mu.get_matches_timelines(match_ids[i][0]))
        timeline.append(mu.get_matches_timelines(match_ids[i][1]))
#         matches.append(match_ids[i][0])
#         timeline.append(match_ids[i][1])
    except:
        continue

100%|██████████████████████████████████████████████████████████████████████████████████████████| 97/97 [00:00<?, ?it/s]


In [100]:
matchs

['KR_6398285495',
 'KR_6404756611',
 'KR_6404247693',
 'KR_6404372851',
 'KR_6404469676',
 'KR_6397412549',
 'KR_6404537011',
 'KR_6404636990',
 'KR_6401643032',
 'KR_6392992699',
 'KR_6404622013',
 'KR_6404152515',
 'KR_6404685902',
 'KR_6403864051',
 'KR_6402716302',
 'KR_6402730786',
 'KR_6403204328',
 'KR_6404771063',
 'KR_6400622622',
 'KR_6404069891',
 'KR_6402373846',
 'KR_6403013657',
 'KR_6398585868',
 'KR_6404627363',
 'KR_6404282877',
 'KR_6404473179',
 'KR_6404348755',
 'KR_6404647189',
 'KR_6404756920',
 'KR_6401952254',
 'KR_6404334381',
 'KR_6401936621',
 'KR_6404708495',
 'KR_6384006005',
 'KR_6403019295',
 'KR_6403614071',
 'KR_6404633537',
 'KR_6402703085',
 'KR_6403923101',
 'KR_6404553323',
 'KR_6399912791',
 'KR_6402711052',
 'KR_6400981175',
 'KR_6403541466',
 'KR_6404221253',
 'KR_6404092655',
 'KR_6401583687',
 'KR_6404764501',
 'KR_6404706772',
 'KR_6402991102',
 'KR_6404287413',
 'KR_6404666181',
 'KR_6403859864',
 'KR_6404067032',
 'KR_6392161815',
 'KR_64046

In [ ]:
get_matches_timelines()

In [9]:
import time

In [10]:
lst = []
for i in tqdm(name_lst[:100]):
    try:
        puuid = mu.get_puuid(i)
        match_ids = mu.get_match_id(puuid,2)
        for match_id in match_ids:
            matches, timeline = mu.get_matches_timelines(match_id)
            lst.append([match_id,matches,timeline])
    except:
        continue
    time.sleep(1)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [04:16<00:00,  2.56s/it]


In [101]:
df = pd.DataFrame(lst,columns=['match_id','matches','timeline'])

In [103]:
df

,match_id,matches,timeline
0,KR_6403658447,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
1,KR_6403651047,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
2,KR_6403306607,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
3,KR_6403244765,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
4,KR_6403851667,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
...,...,...,...
140,status,"{'status': {'message': 'Rate limit exceeded', ...","{'status': {'message': 'Rate limit exceeded', ..."
141,status,"{'status': {'message': 'Rate limit exceeded', ...","{'status': {'message': 'Rate limit exceeded', ..."
142,status,"{'status': {'message': 'Rate limit exceeded', ...","{'status': {'message': 'Rate limit exceeded', ..."
143,status,"{'status': {'message': 'Rate limit exceeded', ...","{'status': {'message': 'Rate limit exceeded', ..."


In [104]:
df_create2 = []

info = df.iloc[0].matches['info']
for i in tqdm(range(len(df))):
    for j in(range(len(info))):
        try:
            df_create1=[]
            df_create1.append(df.iloc[i].matches['info']['gameId'])
            df_create1.append(df.iloc[i].matches['info']['gameDuration'])
            df_create1.append(df.iloc[i].matches['info']['gameVersion'])
            df_create1.append(df.iloc[i].matches['info']['participants'][j]['summonerName'])
            df_create1.append(df.iloc[i].matches['info']['participants'][j]['summonerLevel'])
            df_create1.append(df.iloc[i].matches['info']['participants'][j]['participantId'])
            df_create1.append(df.iloc[i].matches['info']['participants'][j]['championName'])
            df_create1.append(df.iloc[i].matches['info']['participants'][j]['champExperience'])
            df_create1.append(df.iloc[i].matches['info']['participants'][j]['teamPosition'])
            df_create1.append(df.iloc[i].matches['info']['participants'][j]['teamId'])
            df_create1.append(df.iloc[i].matches['info']['participants'][j]['win'])
            df_create1.append(df.iloc[i].matches['info']['participants'][j]['kills'])
            df_create1.append(df.iloc[i].matches['info']['participants'][j]['deaths'])
            df_create1.append(df.iloc[i].matches['info']['participants'][j]['assists'])
            df_create1.append(df.iloc[i].matches['info']['participants'][j]['totalDamageDealtToChampions'])
            df_create1.append(df.iloc[i].matches['info']['participants'][j]['totalDamageTaken'])
            df_create2.append(df_create1)
        except:
            continue

100%|███████████████████████████████████████████████████████████████████████████████| 145/145 [00:01<00:00, 102.14it/s]


In [105]:
match_df = pd.DataFrame(df_create2, columns=['gameId','gameDuration','gameVersion','summonerName','summonerLevel'
                                             ,'participantId','championName','champExperience','teamPosition'
                                             ,'teamId','win','kills','deaths','assists','totalDamageDealtToChampions','totalDamageTaken'])

In [106]:
match_df

,gameId,gameDuration,gameVersion,summonerName,summonerLevel,participantId,championName,champExperience,teamPosition,teamId,win,kills,deaths,assists,totalDamageDealtToChampions,totalDamageTaken
0,6403658447,1503,13.5.495.8836,노블레스혐오자,97,1,Malphite,11633,TOP,100,False,2,9,12,16461,24010
1,6403658447,1503,13.5.495.8836,민채 젖통 핥짝,303,2,Sylas,10122,JUNGLE,100,False,7,10,12,17936,33189
2,6403658447,1503,13.5.495.8836,위대장,584,3,Yasuo,9957,MIDDLE,100,False,4,8,5,8364,18361
3,6403658447,1503,13.5.495.8836,세 가족 행복하게,245,4,Varus,9265,BOTTOM,100,False,9,9,9,22064,19659
4,6403658447,1503,13.5.495.8836,Aoxt,224,5,Senna,9484,UTILITY,100,False,6,8,17,21742,18290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
925,6403957082,1153,13.5.495.8836,성충이 해적단,667,6,Xayah,6116,BOTTOM,200,False,5,4,1,9456,8797
926,6403957082,1153,13.5.495.8836,밍징몽,489,7,Diana,6567,JUNGLE,200,False,1,7,2,10391,18509
927,6403957082,1153,13.5.495.8836,낭인의 검,542,8,Yasuo,8142,MIDDLE,200,False,2,6,1,5631,12879
928,6403957082,1153,13.5.495.8836,고땡9,383,9,Jayce,7819,TOP,200,False,2,7,0,10694,13390


In [38]:
df.iloc[0].matches['info']['participants'][0]['summonerName']

'노블레스혐오자'

In [107]:
m_query = """
CREATE TABLE LOL_MATCHES (gameId bigint(20), gameDuration int(10), gameVersion varchar(20), summonerName varchar(30), 
summonerLevel int(10), participantId int(3), championName varchar(20), champExperience int(10), teamPosition varchar(20),
teamId int(10), win varchar(10), kills int(5), deaths int(5), assists int(5), totalDamageDealtToChampions int(10),
totalDamageTaken int(10))
"""

# my_sql

In [108]:
conn = mu.connect_mysql('icia')
mu.mysql_execute(m_query,conn)
conn.close()


In [109]:
tqdm.pandas()

In [110]:
def insert(x):
    m_query1 =( 
    f'insert into LOL_MATCHES(gameId,gameDuration,gameVersion,summonerName,'
    f'summonerLevel,participantId,championName,champExperience,teamPosition,'
    f'teamId, win, kills, deaths, assists, totalDamageDealtToChampions, totalDamageTaken)'
    f'values({x.gameId},{x.gameDuration},{repr(x.gameVersion)},{repr(x.summonerName)},'
    f'{x.summonerLevel},{x.participantId},{repr(x.championName)},{x.champExperience},{repr(x.teamPosition)},'
    f'{x.teamId},{repr(str(x.win))},{x.kills},{x.deaths},{x.assists},{x.totalDamageDealtToChampions},{x.totalDamageTaken})'
    )
    mu.mysql_execute(m_query1,conn)
    return

In [111]:
mu.db_open()
conn = mu.connect_mysql('icia')
match_df.progress_apply(lambda x: insert(x), axis=1)

oracle open!


100%|██████████████████████████████████████████████████████████████████████████████| 930/930 [00:00<00:00, 2371.65it/s]


0      None
1      None
2      None
3      None
4      None
       ... 
925    None
926    None
927    None
928    None
929    None
Length: 930, dtype: object

In [112]:
query3 = 'select * from lol_matches'

In [113]:
set = pd.DataFrame(mu.mysql_execute_dict(query3,conn))

In [114]:
set

,gameId,gameDuration,gameVersion,summonerName,summonerLevel,participantId,championName,champExperience,teamPosition,teamId,win,kills,deaths,assists,totalDamageDealtToChampions,totalDamageTaken
0,6403658447,1503,13.5.495.8836,노블레스혐오자,97,1,Malphite,11633,TOP,100,False,2,9,12,16461,24010
1,6403658447,1503,13.5.495.8836,민채 젖통 핥짝,303,2,Sylas,10122,JUNGLE,100,False,7,10,12,17936,33189
2,6403658447,1503,13.5.495.8836,위대장,584,3,Yasuo,9957,MIDDLE,100,False,4,8,5,8364,18361
3,6403658447,1503,13.5.495.8836,세 가족 행복하게,245,4,Varus,9265,BOTTOM,100,False,9,9,9,22064,19659
4,6403658447,1503,13.5.495.8836,Aoxt,224,5,Senna,9484,UTILITY,100,False,6,8,17,21742,18290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
925,6403957082,1153,13.5.495.8836,성충이 해적단,667,6,Xayah,6116,BOTTOM,200,False,5,4,1,9456,8797
926,6403957082,1153,13.5.495.8836,밍징몽,489,7,Diana,6567,JUNGLE,200,False,1,7,2,10391,18509
927,6403957082,1153,13.5.495.8836,낭인의 검,542,8,Yasuo,8142,MIDDLE,200,False,2,6,1,5631,12879
928,6403957082,1153,13.5.495.8836,고땡9,383,9,Jayce,7819,TOP,200,False,2,7,0,10694,13390


In [115]:
conn.commit()

In [118]:
df

,match_id,matches,timeline
0,KR_6403658447,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
1,KR_6403651047,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
2,KR_6403306607,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
3,KR_6403244765,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
4,KR_6403851667,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
...,...,...,...
140,status,"{'status': {'message': 'Rate limit exceeded', ...","{'status': {'message': 'Rate limit exceeded', ..."
141,status,"{'status': {'message': 'Rate limit exceeded', ...","{'status': {'message': 'Rate limit exceeded', ..."
142,status,"{'status': {'message': 'Rate limit exceeded', ...","{'status': {'message': 'Rate limit exceeded', ..."
143,status,"{'status': {'message': 'Rate limit exceeded', ...","{'status': {'message': 'Rate limit exceeded', ..."


In [119]:
df.iloc[0].timeline['info'].keys()

dict_keys(['frameInterval', 'frames', 'gameId', 'participants'])

In [121]:
len(df.iloc[0].timeline['info']['frames'])
#27분까지 겜 함

27

In [131]:
df.iloc[0].timeline['info']['frames'][11]['participantFrames']['1']['totalGold']
#1번째 사람 골드
#11분대 스테이터스

3143

In [144]:
df.iloc[0].timeline['info']['frames'][10]['events']
#10~11분 까지

[{'itemId': 1028,
  'participantId': 1,
  'timestamp': 542272,
  'type': 'ITEM_PURCHASED'},
 {'level': 8, 'participantId': 8, 'timestamp': 544604, 'type': 'LEVEL_UP'},
 {'levelUpType': 'NORMAL',
  'participantId': 8,
  'skillSlot': 3,
  'timestamp': 545868,
  'type': 'SKILL_LEVEL_UP'},
 {'level': 6, 'participantId': 4, 'timestamp': 547602, 'type': 'LEVEL_UP'},
 {'levelUpType': 'NORMAL',
  'participantId': 4,
  'skillSlot': 4,
  'timestamp': 548569,
  'type': 'SKILL_LEVEL_UP'},
 {'level': 6, 'participantId': 2, 'timestamp': 551404, 'type': 'LEVEL_UP'},
 {'creatorId': 5,
  'timestamp': 552137,
  'type': 'WARD_PLACED',
  'wardType': 'SIGHT_WARD'},
 {'levelUpType': 'NORMAL',
  'participantId': 2,
  'skillSlot': 4,
  'timestamp': 552569,
  'type': 'SKILL_LEVEL_UP'},
 {'killerId': 4,
  'laneType': 'BOT_LANE',
  'position': {'x': 13866, 'y': 4505},
  'teamId': 200,
  'timestamp': 557265,
  'type': 'TURRET_PLATE_DESTROYED'},
 {'itemId': 2055,
  'participantId': 10,
  'timestamp': 557864,
  'ty

In [192]:
df.iloc[0].timeline['info']['participants'][0]['participantId']

1

In [194]:
lst = []
for i in range(10):
    tmp = []
    tmp.append(df.iloc[0].match_id)
    tmp.append(df.iloc[0].timeline['info']['participants'][i]['participantId'])
    for k in range(5,16):
        tmp.append(df.iloc[0].timeline['info']['frames'][k]['participantFrames'][str(i+1)]['totalGold'])
    lst.append(tmp)                           
                                   

In [197]:
columns = ['match_id','participantsId','g5','g6','g7','g8','g9','g10','g11','g12','g13','g14','g15']

In [198]:
df3 = pd.DataFrame(lst,columns=columns)

In [199]:
df3

,match_id,participantsId,g5,g6,g7,g8,g9,g10,g11,g12,g13,g14,g15
0,KR_6403658447,1,1403,1573,2205,2404,2625,2943,3143,3501,3796,4300,4623
1,KR_6403658447,2,1363,1490,1717,2590,2897,3154,3426,3677,4195,4352,5034
2,KR_6403658447,3,1519,1807,2054,2402,2702,3008,3223,3534,4147,4488,4919
3,KR_6403658447,4,1727,2217,2673,3008,3764,4359,4671,5254,7595,7802,8613
4,KR_6403658447,5,1476,1775,2163,2636,2916,3293,4102,4563,5355,5530,5798
5,KR_6403658447,6,1616,2028,2306,2933,3256,3612,4068,4754,5897,6103,6677
6,KR_6403658447,7,1967,2314,2866,3134,3651,4304,4900,6165,6352,6979,7132
7,KR_6403658447,8,2002,2243,2484,3117,3506,3838,4308,5349,5584,6043,6406
8,KR_6403658447,9,1864,2137,2710,3397,3802,4423,5031,5467,5639,6151,6607
9,KR_6403658447,10,1716,1938,2386,2744,3179,3460,3819,4106,4296,4557,4743
